In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import data_utils as data_utils

dataset = pd.read_csv("datasets/dataset.csv")

print(dataset.shape)
dataset.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,11,15,18,19,20,21,22,23,24,28,30,31,32,33,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2375371, 46)


,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,PET_MPIO,PET_TIPOPER,PQR_CANAL,PQR_CLASE_SNS,PQR_ESTADO,PQR_GRUPOALERTA,PQR_TIPOATENCION,PQR_TIPOPETICION,RIESGO_VIDA,TRIM
0,bogota d.c.,de 13 a 17 años,ninguno,mujer,no aplica,bogotá,otro,no aplica,subsidiado,natural,...,bogotá,juridica,escrito,pqr,cerrado,0,peticion directa,reclamo,no,trim i
1,nariño,de 0 a 5 años,ninguno,hombre,no aplica,puerres,otro,no aplica,subsidiado,natural,...,puerres,natural,telefonico,pqr,cerrado,0,0,reclamo,no,trim i
2,valle,de 13 a 17 años,secundaria,hombre,no aplica,cali,abuelo (a),no aplica,contributivo,natural,...,cali,natural,telefonico,pqr,cerrado,0,0,reclamo,no,trim i
3,huila,de 13 a 17 años,universitario incompleto,hombre,no aplica,neiva,padre,no aplica,contributivo,natural,...,neiva,natural,personalizado,pqr,cerrado,0,0,reclamo,no,trim i
4,risaralda,de 13 a 17 años,ninguno,hombre,no aplica,dosquebradas,padre,persona en condición de discapacidad,contributivo,natural,...,dosquebradas,natural,personalizado,pqr,pendiente (avance),0,0,reclamo,si,trim i


In [2]:
import numpy as np

for column in dataset.columns:
    dataset[column] = dataset[column].apply(lambda value: np.NaN if value == 0 or value == '0' or value == 'nan' else value)     

dataset = dataset[dataset['RIESGO_VIDA'].notnull()]

dataset['CIE_10'] = dataset['CIE_10'].apply(lambda value: np.NaN  if value == 0 or value == '0' or value == 'nan' else value)     
dataset = dataset[dataset['CIE_10'].notnull()]

most_missing_cols = set(dataset.columns[dataset.isna().mean() > 0.75])
dataset = dataset.drop(most_missing_cols, axis = 1)

print(dataset.shape)
set(dataset['RIESGO_VIDA'].values)

(281311, 43)


{'no', 'si'}

In [3]:
dataset = dataset.drop(['IDRANGOEDADES', 'ID_MES', 'PQR_ESTADO'], axis = 1)

dataset['AFEC_DPTO']  = dataset['AFEC_DPTO'].apply(lambda s: 'san andres' if s == 'archipielago de san andres, providencia y santa catalina' or s == 'san andrés' else s)
dataset['AFEC_DPTO']  = dataset['AFEC_DPTO'].apply(lambda s: 'bogota d.c.' if s == 'bogota d.c' else s)
dataset['PET_DPTO']  = dataset['PET_DPTO'].apply(lambda s: 'san andres' if s == 'archipielago de san andres, providencia y santa catalina' or s == 'san andrés' else s)
dataset['PET_DPTO']  = dataset['PET_DPTO'].apply(lambda s: 'bogota d.c.' if s == 'bogota d.c' else s)

dataset = dataset.drop(['ENT_DPTO', 'ENT_MPIO', 'PET_DPTO', 'MACROMOTIVO', 'MOTIVO_GENERAL', 'MOTIVO_ESPECIFICO'], axis = 1)

dataset.head()

,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,PATOLOGIA_1,PATOLOGIA_TIPO,PET_COD_DEPTO,PET_MPIO,PET_TIPOPER,PQR_CANAL,PQR_CLASE_SNS,PQR_TIPOPETICION,RIESGO_VIDA,TRIM
0,bogota d.c.,de 13 a 17 años,ninguno,mujer,no aplica,bogotá,otro,no aplica,subsidiado,natural,...,problemas relacionados con facilidades de aten...,problemas relacionados con facilidades de aten...,11.0,bogotá,juridica,escrito,pqr,reclamo,no,trim i
2,valle,de 13 a 17 años,secundaria,hombre,no aplica,cali,abuelo (a),no aplica,contributivo,natural,...,problemas relacionados con facilidades de aten...,problemas relacionados con facilidades de aten...,76.0,cali,natural,telefonico,pqr,reclamo,no,trim i
3,huila,de 13 a 17 años,universitario incompleto,hombre,no aplica,neiva,padre,no aplica,contributivo,natural,...,problemas relacionados con facilidades de aten...,problemas relacionados con facilidades de aten...,41.0,neiva,natural,personalizado,pqr,reclamo,no,trim i
4,risaralda,de 13 a 17 años,ninguno,hombre,no aplica,dosquebradas,padre,persona en condición de discapacidad,contributivo,natural,...,enfermedaes neurologicas,epilepsia,66.0,dosquebradas,natural,personalizado,pqr,reclamo,si,trim i
6,bogota d.c.,de 13 a 17 años,ninguno,hombre,no aplica,bogotá,otro,no aplica,subsidiado,natural,...,problemas relacionados con facilidades de aten...,problemas relacionados con facilidades de aten...,11.0,bogotá,juridica,escrito,pqr,reclamo,no,trim i


In [4]:
zero_values = list(dataset.columns[dataset.isna().mean() > 0])
dataset[zero_values] = dataset[zero_values].apply(lambda col: col.fillna(col.mode()[0]), axis=0)
dataset = dataset.applymap(str)


## Encoding dataset

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
for column in dataset.columns:
    le.fit(dataset[column])
    dataset[column] = le.transform(dataset[column]) 

dataset.head()

,AFEC_DPTO,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,...,PATOLOGIA_1,PATOLOGIA_TIPO,PET_COD_DEPTO,PET_MPIO,PET_TIPOPER,PQR_CANAL,PQR_CLASE_SNS,PQR_TIPOPETICION,RIESGO_VIDA,TRIM
0,4,1,0,1,4,122,11,3,4,1,...,16,367,0,120,0,2,0,3,0,0
2,30,1,4,0,4,160,0,3,0,1,...,16,367,22,158,1,4,0,3,0,0
3,17,1,7,0,4,675,12,3,0,1,...,16,367,10,654,1,3,0,3,0,0
4,25,1,0,0,4,321,12,4,0,1,...,13,178,18,309,1,3,0,3,1,0
6,4,1,0,0,4,122,11,3,4,1,...,16,367,0,120,0,2,0,3,0,0


## Store processed dataset

In [6]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


dataset.to_csv("datasets/impute_dataset.csv", index = False)